In [1]:
#Importing Data from the CSV file
%matplotlib inline
import string
import math
import numpy as np

f_train = open("training.1600000.processed.noemoticon.csv", "r", encoding="ISO-8859-1")
X_train = f_train.readlines()
f_test = open("testdata.manual.2009.06.14.csv", "r")
X_test = f_test.readlines()

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
#class_0 = number of classes with label = 0
#class_4 = number of classes with label = 4

class_0=class_4=0
tweets_train =[]
for x in X_train:
    a = x.split('","')
    if (a[0] == '"0'): class_0+=1
    else: class_4+=1
    tweets_train.append('%s"|"%s' % (a[0],a[-1]))

#Creating the List with just the tweets (TEST DATA)
tweets_test =[]
for x in X_test:
    a = x.split('","')
    tweets_test.append('%s"|"%s' % (a[0], a[-1]))
    
print("The length of the training set is = ", len(X_train))
print("The number of classes (label=4) =", class_4)
print("The number of classes (label=0) =", class_0)
print("The length of the test set is = ", len(X_test))

f_test.close()
f_train.close()

The length of the training set is =  1600000
The number of classes (label=4) = 800000
The number of classes (label=0) = 800000
The length of the test set is =  359


In [2]:
#Writing the tweets to a file (not necessary - REMOVE if not used anywhere later)
!rm -rf tweet_test.txt
!rm -rf tweet_train.txt
f = open("tweet_train.txt", "w")
for x in tweets_train:
    f.write(x)
f.close()

f = open("tweet_test.txt", "w")
for x in tweets_test:
    f.write(x)
f.close()

In [3]:
def create_dict(tweets_train):
    dictionary = {}
    dict_0 = {}
    dict_4 = {}
    n = []
    for x in tweets_train:
        x = x.rstrip("\n\r")
        x = x.split("|")
        twt = x[1].translate(str.maketrans('','',string.punctuation)).lower()
        words = twt.split() # if I use split(" "), it takes more than one white space as a word
        n.append(len(words))
        for w in words:
    #Global Vocabulary
            if w in dictionary:
                dictionary[w]+=1
            else:
                dictionary[w]=1
    #build Vocabulary for class 0 (-ve class)
            if (x[0] == '"0"'):
                if w in dict_0:
                    dict_0[w]+=1
                else:
                    dict_0[w]=1
    #build Vocabulary for class 4 (+ve class)
            else:
                if w in dict_4:
                    dict_4[w]+=1
                else:
                    dict_4[w]=1
    return dictionary,dict_0,dict_4,n;

In [4]:
dictionary, dict_0, dict_4, len_tweets = create_dict(tweets_train)

In [5]:
m= len(X_train)
phi_0 = class_0/m
phi_4 = class_4/m
v = len(dictionary)

In [6]:
def train_nb_classifier(dictionary, dict_0, dict_4,len_tweets, class_0, class_4):
    theta_0 = {}
    theta_4 = {}
    n_0 = sum(len_tweets[0:class_0])
    n_4 = sum(len_tweets[class_0:len(len_tweets)])
    c=1
    for word in dictionary.keys():
            if word in dict_0:
                theta_0[word] = ((dict_0[word]+c)/(n_0 + v*c))
            else:
                theta_0[word] = ((c) / (n_0 + v*c))
            if word in dict_4:
                theta_4[word] = ((dict_4[word]+c)/(n_4 + v*c))
            else:
                theta_4[word] = ((c)/(n_4 + v*c))
    return theta_0,theta_4;

In [7]:
theta_0, theta_4 = train_nb_classifier(dictionary, dict_0, dict_4, len_tweets, class_0, class_4)

In [8]:
def test_data(tweets):
    pred_class=[]
    actual_class=[]
    for x in tweets:
        test_class0=test_class4=0
        x = x.rstrip("\n\r")
        x = x.split("|")
        twt = x[1].translate(str.maketrans('','',string.punctuation)).lower()
        words = twt.split() # if I use split(" "), it takes more than one white space as a word
        
        #Store actual class number in Test Data for forming Confusion matrix
        actual_class.append(x[0])
        
        #Finding probability of tweet being in a class         
        for w in words:
            if w in theta_0: test_class0 += math.log(theta_0[w])
            else: test_class0 += math.log(1)
            if w in theta_4: test_class4 += math.log(theta_4[w])
            else: test_class4 += math.log(1)
        test_class0 += math.log(phi_0)
        test_class4 += math.log(phi_4)
        #Classifying the probability into classes
        if (test_class0 > test_class4): pred_class.append('"0"')
        else: pred_class.append('"4"')

            
    actual_class=[0 if x=='"0"' else 1 for x in actual_class]
    pred_class=[0 if x=='"0"' else 1 for x in pred_class]
    test_error = sum(np.bitwise_xor(actual_class, pred_class))
    accuracy = ((len(tweets) - test_error)/len(tweets))*100
    return accuracy, actual_class, pred_class;

In [9]:
train_accuracy,actual_class_train,pred_class_train = test_data(tweets_train)
print("Result (a) : The train accuracy of the model on raw data is = {}%".format(train_accuracy))

Result (a) : The train accuracy of the model on raw data is = 84.210875%


In [10]:
test_accuracy, actual_class,pred_class = test_data(tweets_test)
print("Result (a) : The test accuracy of the model on raw data is = {}%".format(test_accuracy))

Result (a) : The test accuracy of the model on raw data is = 80.77994428969359%


In [11]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
#nltk.download()

[nltk_data] Downloading package stopwords to /home/ayushi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ayushi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
tweets_train_pd = pd.Series(tweets_train)

In [15]:
print(tweets_train_pd)

0          "0"|"@switchfoot http://twitpic.com/2y1zl - Aw...
1          "0"|"is upset that he can't update his Faceboo...
2          "0"|"@Kenichan I dived many times for the ball...
3          "0"|"my whole body feels itchy and like its on...
4          "0"|"@nationwideclass no, it's not behaving at...
                                 ...                        
1599995    "4"|"Just woke up. Having no school is the bes...
1599996    "4"|"TheWDB.com - Very cool to hear old Walt i...
1599997    "4"|"Are you ready for your MoJo Makeover? Ask...
1599998    "4"|"Happy 38th Birthday to my boo of alll tim...
1599999    "4"|"happy #charitytuesday @theNSPCC @SparksCh...
Length: 1600000, dtype: object


In [17]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
tweets_train_pd['tokenize_data'] = tweets_train_pd.apply(lambda x: '\n'.join([' '.join(token) for token in ([tknzr.tokenize(x.split("|")[1])])]))

In [20]:
print(tweets_train_pd['clean_data'])

0          [httptwitpiccom2y1zl, awww, thats, a, bummer, ...
1          [is, upset, that, he, cant, update, his, faceb...
2          [i, dived, many, times, for, the, ball, manage...
3          [my, whole, body, feels, itchy, and, like, its...
4          [no, its, not, behaving, at, all, im, mad, why...
                                 ...                        
1599995    [just, woke, up, having, no, school, is, the, ...
1599996    [thewdbcom, very, cool, to, hear, old, walt, i...
1599997    [are, you, ready, for, your, mojo, makeover, a...
1599998    [happy, 38th, birthday, to, my, boo, of, alll,...
1599999                              [happy, charitytuesday]
Length: 1600000, dtype: object


In [19]:
tweets_train_pd['tokenize_data']=tweets_train_pd['tokenize_data'].str.rstrip("\n\r")
tweets_train_pd['clean_data'] = tweets_train_pd['tokenize_data'].str.translate(str.maketrans('','',string.punctuation)).str.lower().str.split()

In [21]:
#Use lambda function to apply the same expression on all the strings together. 
#function takes one string at a time and returns a string wihtout stop words
stop_words=set(stopwords.words('english'))
tweets_train_pd['no_stop_words'] = tweets_train_pd['clean_data'].apply(lambda x: [word for word in x if word not in stop_words])

In [22]:
print(tweets_train_pd['no_stop_words'])

0          [httptwitpiccom2y1zl, awww, thats, bummer, sho...
1          [upset, cant, update, facebook, texting, might...
2          [dived, many, times, ball, managed, save, 50, ...
3                    [whole, body, feels, itchy, like, fire]
4                             [behaving, im, mad, cant, see]
                                 ...                        
1599995                  [woke, school, best, feeling, ever]
1599996    [thewdbcom, cool, hear, old, walt, interviews,...
1599997                [ready, mojo, makeover, ask, details]
1599998    [happy, 38th, birthday, boo, alll, time, tupac...
1599999                              [happy, charitytuesday]
Length: 1600000, dtype: object


In [23]:
wordnet_lemmatizer = WordNetLemmatizer()
tweets_train_pd['lemma1'] = tweets_train_pd['no_stop_words'].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos ="n") for word in x])
tweets_train_pd['lemma2'] = tweets_train_pd['lemma1'].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos ="v") for word in x])
tweets_train_pd['lemma'] = tweets_train_pd['lemma2'].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos ="a") for word in x])

In [24]:
print(tweets_train_pd['lemma'])

0          [httptwitpiccom2y1zl, awww, thats, bummer, sho...
1          [upset, cant, update, facebook, texting, might...
2          [dive, many, time, ball, manage, save, 50, res...
3                     [whole, body, feel, itchy, like, fire]
4                               [behave, im, mad, cant, see]
                                 ...                        
1599995                     [wake, school, best, feel, ever]
1599996    [thewdbcom, cool, hear, old, walt, interview, ...
1599997                 [ready, mojo, makeover, ask, detail]
1599998    [happy, 38th, birthday, boo, alll, time, tupac...
1599999                              [happy, charitytuesday]
Length: 1600000, dtype: object
